# Explorative Data Analysis (EDA) – Online Retail Dataset

Dieses Notebook analysiert das **Online Retail Dataset** auf Basis einer performanten
Parquet-Datei (`retail_raw.parquet`).

Ziele dieses Notebooks:
- Verständnis der Datenstruktur und Datentypen
- Prüfung der Datenqualität (Missing Values, ungültige Werte, Duplikate)
- Ableitung erster betriebswirtschaftlicher Kennzahlen (KPIs)
- Vorbereitung für weiterführende Analysen (Customer Analytics, Time Series, ML)

Die Analyse erfolgt **realitätsnah**, d. h. inklusive typischer Probleme wie Retouren,
fehlenden Kundennummern und fehlerhaften Preisen.


## 1. Laden der Daten aus Parquet

In dieser Zelle wird das vorbereitete **Parquet-File** geladen.
Parquet ist ein spaltenorientiertes, performantes Speicherformat und eignet sich
besonders für analytische Workflows.

Ziel:
- Sicherstellen, dass die Daten korrekt geladen werden
- Erste visuelle Prüfung der Datensätze


In [2]:
from pathlib import Path
import pandas as pd

# Projekt-Root ausgehend vom Notebook-Pfad bestimmen
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()

DATA_PATH = PROJECT_ROOT / "data" / "processed" / "retail_raw.parquet"

print("CWD:", Path.cwd())
print("PROJECT_ROOT:", PROJECT_ROOT)
print("DATA_PATH exists:", DATA_PATH.exists())
print("DATA_PATH:", DATA_PATH)

df = pd.read_parquet(DATA_PATH)
df.head()


CWD: c:\Users\admin\Desktop\AI Sec Project\GitHub\data-science-projects\1.1-ecommerce-analytics\notebooks
PROJECT_ROOT: c:\Users\admin\Desktop\AI Sec Project\GitHub\data-science-projects\1.1-ecommerce-analytics
DATA_PATH exists: True
DATA_PATH: c:\Users\admin\Desktop\AI Sec Project\GitHub\data-science-projects\1.1-ecommerce-analytics\data\processed\retail_raw.parquet


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


## 2. Struktur- und Typübersicht

Hier wird ein erster struktureller Überblick über den Datensatz erzeugt:

- Anzahl der Zeilen und Spalten
- Datentypen jeder Spalte
- Statistische Basisinformationen (auch für kategoriale Spalten)

Dies ist ein zentraler Schritt, um:
- falsche Datentypen zu erkennen
- Auffälligkeiten früh zu identifizieren


In [3]:
print("shape:", df.shape)
display(df.dtypes)
display(df.describe(include="all").T.head(20))


shape: (541909, 8)


InvoiceNo                 str
StockCode                 str
Description               str
Quantity                int64
InvoiceDate    datetime64[us]
UnitPrice             float64
CustomerID              Int64
Country                   str
dtype: object

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
InvoiceNo,541909,25900,573585,1114,NaN,NaN,NaN,NaN,NaN,NaN,NaN
StockCode,541909,4070,85123A,2313,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Description,541909,4224,WHITE HANGING HEART T-LIGHT HOLDER,2369,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Quantity,541909.0,NaN,NaN,NaN,9.55225,-80995.0,1.0,3.0,10.0,80995.0,218.081158
InvoiceDate,541909,NaN,NaN,NaN,2011-07-04 13:34:57.156386,2010-12-01 08:26:00,2011-03-28 11:34:00,2011-07-19 17:17:00,2011-10-19 11:27:00,2011-12-09 12:50:00,NaN
UnitPrice,541909.0,NaN,NaN,NaN,4.611114,-11062.06,1.25,2.08,4.13,38970.0,96.759853
CustomerID,406829.0,<NA>,<NA>,<NA>,15287.69057,12346.0,13953.0,15152.0,16791.0,18287.0,1713.600303
Country,541909,38,United Kingdom,495478,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 3. Analyse fehlender Werte

Fehlende Werte sind in realen Datensätzen normal und müssen bewusst behandelt werden.

In dieser Zelle wird:
- der Anteil fehlender Werte pro Spalte berechnet
- sichtbar gemacht, welche Variablen kritisch sind

Beispiel:
- Fehlende `CustomerID` → anonyme Käufe
- Fehlende Texte → mögliche Datenqualitätsprobleme


In [4]:
missing = df.isna().mean().sort_values(ascending=False)
display(missing)


CustomerID     0.249267
InvoiceNo      0.000000
Description    0.000000
StockCode      0.000000
Quantity       0.000000
InvoiceDate    0.000000
UnitPrice      0.000000
Country        0.000000
dtype: float64

## 4. Datenqualitätsprüfungen (Retail-spezifisch)

Diese Prüfungen orientieren sich an typischen Problemen im Einzelhandel:

- Negative Mengen → Retouren oder Stornierungen
- Preise ≤ 0 → fehlerhafte Buchungen
- Fehlende CustomerID → Gastkäufe
- Duplikate → Mehrfachimporte oder Systemfehler

Die Ergebnisse helfen zu entscheiden, welche Daten
für Umsatzanalysen geeignet sind.


In [5]:
checks = {
    "negative_quantity": (df["Quantity"] < 0).mean(),
    "zero_or_negative_unitprice": (df["UnitPrice"] <= 0).mean(),
    "missing_customerid": df["CustomerID"].isna().mean(),
    "duplicate_rows": df.duplicated().mean(),
}
pd.Series(checks).sort_values(ascending=False)


missing_customerid            0.249267
negative_quantity             0.019605
duplicate_rows                0.009721
zero_or_negative_unitprice    0.004645
dtype: float64

## 5. Umsatzberechnung und Filterung gültiger Verkäufe

Der Umsatz wird als:
> Quantity × UnitPrice

definiert.

Für betriebswirtschaftliche Analysen werden **nur echte Verkäufe** berücksichtigt:
- Menge > 0
- Preis > 0

Retouren und fehlerhafte Buchungen werden ausgeschlossen,
bleiben aber für spätere Analysen erhalten.


In [6]:
df["Revenue"] = df["Quantity"] * df["UnitPrice"]

# "Sales" = nur echte Verkäufe (keine Returns, keine 0/neg prices)
sales = df[(df["Quantity"] > 0) & (df["UnitPrice"] > 0)].copy()
sales["Revenue"] = sales["Quantity"] * sales["UnitPrice"]

sales[["InvoiceNo","InvoiceDate","Country","CustomerID","Revenue"]].head()


,InvoiceNo,InvoiceDate,Country,CustomerID,Revenue
0,536365,2010-12-01 08:26:00,United Kingdom,17850,15.30
1,536365,2010-12-01 08:26:00,United Kingdom,17850,20.34
2,536365,2010-12-01 08:26:00,United Kingdom,17850,22.00
3,536365,2010-12-01 08:26:00,United Kingdom,17850,20.34
4,536365,2010-12-01 08:26:00,United Kingdom,17850,20.34


## 6. Zentrale Geschäftskennzahlen (KPIs)

In dieser Zelle werden grundlegende KPIs berechnet:

- Anzahl der Bestellungen
- Anzahl eindeutiger Kunden
- Anzahl belieferten Länder
- Gesamtumsatz
- Durchschnittlicher Bestellwert (Average Order Value)

Diese Kennzahlen bilden die Basis für Management-Reports
und strategische Entscheidungen.


In [7]:
kpis = {
    "orders": sales["InvoiceNo"].nunique(),
    "customers": sales["CustomerID"].nunique(dropna=True),
    "countries": sales["Country"].nunique(),
    "total_revenue": float(sales["Revenue"].sum()),
    "avg_order_value": float(sales.groupby("InvoiceNo")["Revenue"].sum().mean()),
}
pd.Series(kpis)


orders             1.996000e+04
customers          4.338000e+03
countries          3.800000e+01
total_revenue      1.066668e+07
avg_order_value    5.344030e+02
dtype: float64

## 7. Umsatzstärkste Länder und Produkte

Zum Abschluss der EDA werden:
- die umsatzstärksten Länder
- die umsatzstärksten Produkte

ermittelt.

Diese Analysen sind besonders relevant für:
- Marktpriorisierung
- Sortimentsentscheidungen
- Internationalisierungsstrategien


In [8]:
top_countries = sales.groupby("Country")["Revenue"].sum().sort_values(ascending=False).head(10)
top_products = sales.groupby(["StockCode","Description"])["Revenue"].sum().sort_values(ascending=False).head(10)

display(top_countries)
display(top_products)


Country
United Kingdom    9025222.084
Netherlands        285446.340
EIRE               283453.960
Germany            228867.140
France             209715.110
Australia          138521.310
Spain               61577.110
Switzerland         57089.900
Belgium             41196.340
Sweden              38378.330
Name: Revenue, dtype: float64

StockCode  Description                       
DOT        DOTCOM POSTAGE                        206248.77
22423      REGENCY CAKESTAND 3 TIER              174484.74
23843      PAPER CRAFT , LITTLE BIRDIE           168469.60
85123A     WHITE HANGING HEART T-LIGHT HOLDER    104340.29
47566      PARTY BUNTING                          99504.33
85099B     JUMBO BAG RED RETROSPOT                94340.05
23166      MEDIUM CERAMIC TOP STORAGE JAR         81700.92
M          Manual                                 78110.27
POST       POSTAGE                                78101.88
23084      RABBIT NIGHT LIGHT                     66964.99
Name: Revenue, dtype: float64